In [ ]:
from selenium import webdriver # 셀레니움과 웹드라이버 부르는 모듈함수 
import time
import pymysql

   
    
    
    
    
# croll 2번씩 내려감
def scroll_dowmn_page2():
    for i in range(2): # adjust integer value for need
        driver.execute_script("window.scrollBy(0, 200)")
        time.sleep(0.5)
    
    
    
    
    

# AWS DB 연동
def get_db():
    db = pymysql.connect(
        host='ec2-13-125-144-25.ap-northeast-2.compute.amazonaws.com',
        port=3306,
        user='admin',
        passwd='1/zw;GytAwx*',
        db='KABL',
        charset='utf8'
    )
    return db







# AWS Inquiry TABLE DB에 데이터 넣기
def insert_inquiry(data):
    db = get_db()
    mycursor = db.cursor()

    try:
        sql = "INSERT INTO Inquiry (Num,Title,regDate,context,OrgIndex,Link) VALUES (%s, %s, %s, %s, %s, %s)"
        val = (data[1],data[2],data[3],data[4],data[0],data[5])
        mycursor.execute(sql, val)

        db.commit()
        db.close()

    except Exception as e:
        print('db insert inquiry error', e)
        db.close()



        

        
# 클릭전에 추출하고 페이지 클릭해야함.

def board_data():
    board = []
    org = 1

    try :
        case_num = driver.find_element_by_xpath(f"//*[@id='table_kjaa']/tbody/tr[{i}]/td[1]").text     # tr[1] ~ tr[10]
        title = driver.find_element_by_xpath(f"//*[@id='table_kjaa']/tbody/tr[{i}]/td[2]/a").text    # tr[1] ~ tr[10]
        regdate = driver.find_element_by_xpath(f"//*[@id='table_kjaa']/tbody/tr[{i}]/td[4]").text

        board.append(org)
        board.append(case_num)
        board.append(title)
        board.append(regdate)


        # 여기서부터 게시글을 클릭하여 데이터내용을가져옴
        driver.implicitly_wait(10)
        board_click = driver.find_element_by_xpath(f"//*[@id='table_kjaa']/tbody/tr[{i}]/td[2]/a").click()    # tr[1] ~ tr[10]
        context = driver.find_element_by_xpath("//*[@id='contents']/div/div/div/div[2]/div[2]").text
        context = context.replace("--------------- 원글 내용 ---------------","")
        context = context.replace("\n","")
        board.append(context)

        link = driver.current_url
        board.append(link)
        driver.back()
        print(board)
        print("\n\n\n")

        insert_inquiry(board)
        return board

            
    except Exception as e:
        print("데이터 추출 Error:",e)

    #return board        
        
        
        
    
# ---------------------------------------- main -------------------------------------------------------------
driver = webdriver.Chrome('C:\/chromedriver.exe') # 제어하기 위해 변수 선언
driver.maximize_window() ### 창의 해상도 크기 최대화 (전체화면) ###


print("==================================================================================")
print("--------------------로컬 PC에서 Crawling 작업을 시작하겠습니다----------------------")
print("==================================================================================")
print("--------------------------------서울중앙지법무사협회--------------------------------")
print("\n\n")



url = "http://lawlandr.kabl.kr/mod/board/list.do?MID=lawland_onlinecns_01"
driver.get(url)
lawlandr_board = 1   # 게시글 수



try :    
    while True :
   
        # 페이지네이션 1->2->3->10
        for page in range(3,13):
            scroll_dowmn_page2()
            print("현재 작업중인 page(3이 1페이지를 뜻함) :",page)
            case = driver.find_element_by_xpath(f"//*[@id='contents']/div/div/div[2]/div[3]/ul/li[{page}]").click()
              
            lawlandr_board += 1
            
            
            # 이건 페이지네이션을 위한 확인 용도임
            #print("********************************************page:",page)
            #print("********************************************lawlandr_board:",lawlandr_board)
            
            # 작업중인 페이지 page 12, lawlandr_board 10이면 다음페이지로 이동 (페이지네이션)
            if page == 12 and lawlandr_board == 11:
                next_page = driver.find_element_by_xpath("//*[@id='contents']/div/div/div[2]/div[3]/ul/li[13]/a").click()
                lawlandr_board = 1   # 게시글 수 초기화 --> 다음페이지로 넘겨주기 위해
                
                
            for i in range(1,11):
                print("현재 작업중인 게시글(1~10) :",i)
                board_data()


# 가끔식 에러가 뜨는데, 속도가 빨라서 그런거임 
except Exception as e:
    print("데이터 로딩 error",e)



--------------------로컬 PC에서 Crawling 작업을 시작하겠습니다----------------------
--------------------------------서울중앙지법무사협회--------------------------------



현재 작업중인 page(3이 1페이지를 뜻함) : 3
현재 작업중인 게시글(1~10) : 1
[1, '20004', '미국 법인이 한국법인 합병 NEW', '2021-07-14', '미국 법인이 한국법인 합병하려고 하는데, 한국에서 신고할때 어떤 서류가 제출되어야 할까요??', 'http://lawlandr.kabl.kr/mod/board/view.do?MID=lawland_onlinecns_01&DOC_NO=162623923182859&page=1']




현재 작업중인 게시글(1~10) : 2
[1, '20003', '└ re. 미국 법인이 한국법인 합병 NEW', '2021-07-14', '미국 법인이 한국법인 합병하려고 하는데, 한국에서 신고할때 어떤 서류가 제출되어야 할까요?? 위 문의에 대한 답변은 먼저회계. 새무관계등이 확인되어야 하여 구체적인 답변이 어려운 점 이해하시가 바랍니다.', 'http://lawlandr.kabl.kr/mod/board/view.do?MID=lawland_onlinecns_01&DOC_NO=162625171120629&page=1']




현재 작업중인 게시글(1~10) : 3
[1, '20002', '문의 드립니다. NEW', '2021-07-12', ' 안녕하세요. 알코올 중독 아버지에 관한 문의 드립니다.  13년 전, 아버지의 무리한 주식 투자로 인한 빚, 폭력 및 살인 위협, 알코올 중독으로 부모님은 이혼하셨습니다. 현재는 오빠와 저도 직장 거리나 결혼으로 인해 4식구가 모두 따로 살고 있습니다. 아버지는 18년부터 알코올 중독으로 인한 치료를 받기 시작하였고, 알코올성 치매로 인해 작년에는 실종, 올해는 이웃에 피해를 끼쳐 강제입원과 퇴원을 반